In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [3]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

In [4]:
customer_id=39597

## Load Data

In [5]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv("LOAD_DATA_FROM_DWH"))

false


In [6]:
data_loader_dwh = DataLoader(load_from_db=False)
data_loader_parquet = DataLoader(load_from_db=False)

In [7]:
ppm_repo = PortpropMatricesRepository(data_loader=data_loader_parquet)
ports_repo = PortfoliosRepository(data_loader=data_loader_dwh)
rebalancer_repo = RebalancerRepository(data_loader=data_loader_parquet)

In [8]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-09-30"
)
client_styles = ports_repo.load_client_style(as_of_date="2025-09-30")

In [9]:
client_styles

,as_of_date,customer_id,client_full_name_th,client_first_name_en,client_last_name_en,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team
0,2025-09-30,6,น*********************************************...,M*********************************************...,<NA>,Bulletproof,Wealth,PWM,Non-qualified,Dormant,208,N,N********,I************
1,2025-09-30,11,น*********************************************...,M*********************************************...,<NA>,Bulletproof,Wealth,PWM,Non-qualified,Dormant,323,Y,N********,R***********
2,2025-09-30,12,น*********************************************...,M*********************************************...,<NA>,Bulletproof,Wealth,PWM,Non-qualified,Dormant,323,Y,N********,R***********
3,2025-09-30,13,น****************************************,M*********************************************...,<NA>,Bulletproof,Wealth,PWM,Non-qualified,Dormant,349,N,P*******,P*************
4,2025-09-30,15,น*********************************************...,M*********************************************...,<NA>,Unwavering,Wealth,PWM,Non-qualified,Unqualified,70,N,T******,K************
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91530,2025-09-30,115586,ว*******************,W********,P*********,Bulletproof,Edge,EDGE,Non-qualified,Dormant,84,N,D***********,D***********
91531,2025-09-30,115587,พ***************,P*********,W*********,Bulletproof,Edge,EDGE,Non-qualified,Dormant,84,N,D***********,D***********
91532,2025-09-30,115588,บ********************,B*******,P************,Bulletproof,Edge,EDGE,Non-qualified,Dormant,84,N,D***********,D***********
91533,2025-09-30,115589,ว************************,V*********,J*************,Bulletproof,Edge,EDGE,Non-qualified,Dormant,84,N,D***********,D***********


In [12]:
ports_ref_table = {
    "product_mapping": ports_repo.load_product_mapping(as_of_date="2025-09-30"),
    "product_underlying": ports_repo.load_product_underlying(),
}

In [13]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [14]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

## Instances

In [15]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(
    client_out_enriched, client_styles, column_mapping=["as_of_date", "customer_id"]
)
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer(
    #client_investment_style="Moderate High Risk",
    client_classification="Non-UI",
    discretionary_acceptance=0,
    new_money=0,
    product_whitelist=["KKP", "PTTEP"],
    product_blacklist=["KKP SIB-USD"],
)
rb.set_ref_tables(rb_ref_dict)

## Portfolio (Service)

In [16]:
## get list of all customer ids
print(port_service.get_all_customer_ids()[0:5])

[26599, 25914, 11300, 26527, 110343]


In [17]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=customer_id)

In [18]:
port.df_style.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 48507 to 48507
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   port_id                        1 non-null      int64 
 1   client_full_name_th            1 non-null      string
 2   client_first_name_en           1 non-null      string
 3   client_last_name_en            1 non-null      string
 4   port_investment_style          1 non-null      string
 5   client_tier                    1 non-null      string
 6   business_unit                  1 non-null      string
 7   client_segment_by_inv_aum      1 non-null      string
 8   client_sub_segment_by_inv_aum  1 non-null      string
 9   sales_id                       1 non-null      string
 10  ui_client                      1 non-null      string
 11  sales_first_name_en            1 non-null      string
 12  sales_team                     1 non-null      string
 13  portpo

In [19]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
268477,39597,2025-09-30,M00127344,KKP CorePath Extra-ES,TRADE,L,THB,KKP CorePath Extra-ES,Mutual Fund,Allocation,...,True,0.078,False,<NA>,Not Top-Pick,<NA>,11848.28,19731,NaN,0.087160
268481,39597,2025-09-30,M00162119,KKP PGE-UH,TRADE,L,THB,KKP PGE-UH,Mutual Fund,Global Equity,...,True,0.077,False,<NA>,Top-Pick,<NA>,13105.20,19731,AA_GE,0.096406
271293,39597,2025-09-30,M00228153,K-GPINUH-A(A),TRADE,L,THB,K-GPINUH-A(A),Mutual Fund,Global Equity,...,True,0.087,False,<NA>,Top-Pick,<NA>,48481.68,19731,AA_GE,0.356647
272207,39597,2025-09-30,M00072598,KFGBRAND-A,TRADE,L,THB,KFGBRAND-A,Mutual Fund,Global Equity,...,True,0.087,False,<NA>,Top-Pick,<NA>,10489.50,19731,AA_GE,0.077164
273884,39597,2025-09-30,C00142044,SS_2013096061,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,True,0.040,False,<NA>,Not Top-Pick,<NA>,0.83,19731,AA_CASH,0.000006
276668,39597,2025-09-30,M00248078,KKP EWUS500-UH,TRADE,L,THB,KKP EWUS500-UH,Mutual Fund,Global Equity,...,True,0.087,False,<NA>,Top-Pick,<NA>,52011.99,19731,AA_GE,0.382617


In [20]:
port.product_mapping.columns

Index(['product_id', 'src_sharecodes', 'desk', 'port_type', 'currency',
       'product_display_name', 'product_type_desc', 'asset_class_name',
       'symbol', 'pp_asset_sub_class', 'is_risky_asset', 'coverage_prdtype',
       'is_coverage', 'expected_return', 'es_core_port', 'es_sell_list',
       'flag_top_pick', 'flag_tax_saving'],
      dtype='object')

In [21]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,19731,0.011331,0.003492,0.006101,0.962515,0.01656


In [22]:
# get model allocation
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,19731,Unwavering,Aggressive,0.105,0.055,0.08,0.684,0.076


In [23]:
port.df_style

,port_id,client_full_name_th,client_first_name_en,client_last_name_en,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team,portpop_styles
48507,19731,ธ******************,T*********,P**********,Unwavering,Edge,EDGE,Qualified,<2mb,84,N,D***********,D***********,Aggressive


## Healthscore

In [24]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm, hs)

In [25]:
health_score["health_score"].values[0]

np.float64(6.0)

In [26]:
health_score_comp

,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,...,ge_other,expected_return,volatility,is_bulk_risk,underlying_company,issure_risk_group,coverage_prdtype,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund
0,19731,M00127344,KKP CorePath Extra-ES,TRADE,L,THB,KKP CorePath Extra-ES,Mutual Fund,Allocation,11848.28,...,0.003510,6.798462e-03,1.115429e-02,False,<NA>,NaN,MUTUAL_FUND,0,0,0
1,19731,M00162119,KKP PGE-UH,TRADE,L,THB,KKP PGE-UH,Mutual Fund,Global Equity,13105.20,...,0.006811,7.423268e-03,1.627748e-02,False,<NA>,NaN,MUTUAL_FUND,0,0,0
2,19731,M00228153,K-GPINUH-A(A),TRADE,L,THB,K-GPINUH-A(A),Mutual Fund,Global Equity,48481.68,...,0.025196,3.102828e-02,6.021727e-02,True,<NA>,NaN,MUTUAL_FUND,0,0,0
3,19731,M00072598,KFGBRAND-A,TRADE,L,THB,KFGBRAND-A,Mutual Fund,Global Equity,10489.50,...,0.005452,6.713281e-03,1.302861e-02,False,<NA>,NaN,MUTUAL_FUND,0,0,0
4,19731,C00142044,SS_2013096061,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,0.83,...,NaN,2.442299e-07,-2.927335e-10,False,<NA>,NaN,N/A,0,0,0
5,19731,M00248078,KKP EWUS500-UH,TRADE,L,THB,KKP EWUS500-UH,Mutual Fund,Global Equity,52011.99,...,0.000000,3.328768e-02,6.492894e-02,True,<NA>,NaN,MUTUAL_FUND,0,0,0


In [27]:
health_score_comp.columns

Index(['port_id', 'product_id', 'src_sharecodes', 'desk', 'port_type',
       'currency', 'product_display_name', 'product_type_desc',
       'asset_class_name', 'value', 'weight', 'aa_alt', 'aa_cash', 'aa_fi',
       'aa_ge', 'aa_le', 'ge_em', 'ge_eur', 'ge_jp', 'ge_us', 'ge_other',
       'expected_return', 'volatility', 'is_bulk_risk', 'underlying_company',
       'issure_risk_group', 'coverage_prdtype', 'score_non_cover_global_stock',
       'score_non_cover_local_stock', 'score_non_cover_mutual_fund'],
      dtype='object')

## Rebalancer

In [28]:
new_port, recommendations = rb.rebalance(port, ppm, hs)

In [29]:
recommendations

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,weight,flag,expected_weight,action,amount,flag_msg
0,1,1,19731,M00228153,K-GPINUH-A(A),TRADE,L,THB,K-GPINUH-A(A),Mutual Fund,Global Equity,48481.68,0.356647,bulk_risk,0.19,sell,-22653.5588,Reduce or exit to manage overconcentration in ...
1,2,1,19731,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,22653.5588,Fund transactions through the cash proxy posit...
2,3,2,19731,M00248078,KKP EWUS500-UH,TRADE,L,THB,KKP EWUS500-UH,Mutual Fund,Global Equity,52011.99,0.382617,bulk_risk,0.19,sell,-26183.8688,Reduce or exit to manage overconcentration in ...
3,4,2,19731,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,26183.8688,Fund transactions through the cash proxy posit...
4,5,3,19731,M00073292,KKP GINFRAEQ-H,TRADE,L,THB,KKP GINFRAEQ-H,Mutual Fund,Alternative,0.00,0.000000,alternative_buy,0.10,buy,13593.7480,Increase allocation to alternative to align th...
5,6,3,19731,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,-13593.7480,Fund transactions through the cash proxy posit...
6,7,4,19731,M00073293,KKP GNP,TRADE,L,THB,KKP GNP,Mutual Fund,Global Equity,0.00,0.000000,global_equity_buy,0.10,buy,13593.7480,Increase allocation to global equity to align ...
7,8,4,19731,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,-13593.7480,Fund transactions through the cash proxy posit...
8,9,5,19731,M00071550,KF-CSINCOME,TRADE,L,THB,KF-CSINCOME,Mutual Fund,Fixed Income,0.00,0.000000,fixed_income_buy,0.10,buy,13593.7480,Increase allocation to fixed income to align t...
9,10,5,19731,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,-13593.7480,Fund transactions through the cash proxy posit...


In [30]:
new_port.df_out

,port_id,product_id,src_sharecodes,desk,port_type,currency,value,product_display_name,product_type_desc,asset_class_name,...,is_risky_asset,coverage_prdtype,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,asset_class_code,weight
0,19731,C00142044,SS_2013096061,TRADE,L,THB,0.8300,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,False,N/A,True,0.040,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.000006
1,19731,CTHB00000000,Cash Proxy THB,TRADE,L,THB,8056.1836,Cash Proxy THB,Cash,Cash and Cash Equivalent,...,False,N/A,True,0.040,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.059264
2,19731,M00071550,KF-CSINCOME,TRADE,L,THB,13593.7480,KF-CSINCOME,Mutual Fund,Fixed Income,...,True,MUTUAL_FUND,False,0.048,False,<NA>,Not Top-Pick,<NA>,AA_FI,0.100000
3,19731,M00072598,KFGBRAND-A,TRADE,L,THB,10489.5000,KFGBRAND-A,Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.087,False,<NA>,Top-Pick,<NA>,AA_GE,0.077164
4,19731,M00073292,KKP GINFRAEQ-H,TRADE,L,THB,13593.7480,KKP GINFRAEQ-H,Mutual Fund,Alternative,...,True,MUTUAL_FUND,True,0.076,False,<NA>,Top-Pick,<NA>,AA_ALT,0.100000
5,19731,M00073293,KKP GNP,TRADE,L,THB,13593.7480,KKP GNP,Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.087,False,<NA>,Top-Pick,<NA>,AA_GE,0.100000
6,19731,M00127344,KKP CorePath Extra-ES,TRADE,L,THB,11848.2800,KKP CorePath Extra-ES,Mutual Fund,Allocation,...,False,MUTUAL_FUND,True,0.078,False,<NA>,Not Top-Pick,<NA>,NaN,0.087160
7,19731,M00162119,KKP PGE-UH,TRADE,L,THB,13105.2000,KKP PGE-UH,Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.077,False,<NA>,Top-Pick,<NA>,AA_GE,0.096406
8,19731,M00228153,K-GPINUH-A(A),TRADE,L,THB,25828.1212,K-GPINUH-A(A),Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.087,False,<NA>,Top-Pick,<NA>,AA_GE,0.190000
9,19731,M00248078,KKP EWUS500-UH,TRADE,L,THB,25828.1212,KKP EWUS500-UH,Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.087,False,<NA>,Top-Pick,<NA>,AA_GE,0.190000


In [31]:
health_score, health_score_comp = new_port.get_portfolio_health_score(ppm, hs)

In [32]:
health_score["health_score"].values[0]

np.float64(10.0)